In [51]:
from helper_load import get_engine_database
from collections import defaultdict
import pandas as pd

with open("./secrets.txt", "r") as f:
    driver, username, password, host, port, database = f.read().split(",")
engine = get_engine_database(driver, username, password, host, port, database)

In [52]:
socios = pd.read_excel("Empresas-Sócios_CNAE-9602501_Cidade-Maringa_test_1.xlsx", sheet_name="Sócios")
socios.head()

,CNPJ Básico,Identificador Sócio,Nome Sócio,CNPJ/CPF Sócio,Qualificação Sócio,Data Entrada Sociedade,País,CPF Representante Legal,Nome Representante Legal
0,24699192,PESSOA FISICA,NATHANE DE SA DOS SANTOS,***167958**,Sócio-Administrador,12/02/2023,NAO DECLARADOS,***000000**,NaN
1,16653244,PESSOA FISICA,FABIANA LEIKO MIKUNI DE FREITAS,***979549**,Sócio-Administrador,27/07/2021,NAO DECLARADOS,***000000**,NaN
2,15126007,PESSOA FISICA,ELEZIER FREITAS DE CASTRO,***013109**,Sócio-Administrador,24/07/2012,NAO DECLARADOS,***000000**,NaN
3,43868178,PESSOA FISICA,RENATO ALEXANDRO DE ANDRADE JUNIOR,***376929**,Sócio-Administrador,14/10/2021,NAO DECLARADOS,***000000**,NaN
4,43868178,PESSOA FISICA,LUCAS FERNANDO FRANZIN,***870719**,Sócio-Administrador,14/10/2021,NAO DECLARADOS,***000000**,NaN


In [53]:
socio_numero = defaultdict(str)
nomes, cnpjs_cpfs = socios["Nome Sócio"], socios["CNPJ/CPF Sócio"]
for nome, cnpj_cpf in zip(nomes, cnpjs_cpfs):
    with engine.begin() as conn:
        qry = f"""SELECT
        telefone
    FROM public.socio_telefone
    WHERE nome_socio = '{nome}' AND cnpj_cpf_socio = '{cnpj_cpf}' AND telefone <> ' ';"""

        socio_numero[f"{nome}-{cnpj_cpf}"] = pd.read_sql(qry, conn).dropna().drop_duplicates()["telefone"].values.tolist()

todos_telefones = set()
for _, t in dict(socio_numero).items():
    if len(t) == 1:
        todos_telefones.add(t[0])
    else:
        for i in t:
            todos_telefones.add(i)

socio_numero = dict(socio_numero)
print(socio_numero)
print(todos_telefones)

In [ ]:
telefones = []
for nome, cnpj_cpf in zip(nomes, cnpjs_cpfs):
    ts = socio_numero[f"{nome}-{cnpj_cpf}"]
    
    t = ts[0]
    if t in todos_telefones:
        telefones.append([nome, cnpj_cpf, t])
        todos_telefones.remove(t)
    else:
        telefones.append("replace")
        
    if len(ts) == 1:
        del socio_numero[f"{nome}-{cnpj_cpf}"]
    else:
        socio_numero[f"{nome}-{cnpj_cpf}"] = ts[1:]

telefones

[['NATHANE DE SA DOS SANTOS', '***167958**', '44 99649820'],
 ['FABIANA LEIKO MIKUNI DE FREITAS', '***979549**', '44 30288724'],
 ['ELEZIER FREITAS DE CASTRO', '***013109**', '44 30297869'],
 ['RENATO ALEXANDRO DE ANDRADE JUNIOR', '***376929**', '44 99640852'],
 ['LUCAS FERNANDO FRANZIN', '***870719**', '44 99640852'],
 ['FELIPE GERALDI', '***628749**', '44 99999999'],
 ['JEISIELE GONCALVES DE OLIVEIRA CARVALHO', '***656699**', '44 30472810'],
 ['JEISILA CRISTIAN GONCALVES DE OLIVEIRA', '***555879**', '44 30472810'],
 ['NILZA DE FATIMA DE OLIVEIRA', '***086529**', '44 30472810'],
 ['RONALDO BALEEIRO VIOTTO', '***477459**', '44 98612552'],
 ['STEFANIE CAMILA DE OLIVEIRA MALTA', '***612599**', '44 98612552'],
 ['FLAVIA LOPES DE POLLI BENITES', '***242959**', '44 98058366'],
 ['LUZIA HERRERA DE CASTRO', '***513199**', '44 30297869'],
 ['GABRIELA PAISCA DA SILVA', '***656769**', '44 99449519'],
 ['DANIELA PAISCA MAGALHAES DA SILVA', '***916339**', '44 99449519'],
 ['ALINE DE ARRUDA GARCIA 

In [ ]:
socio_numero

{'FELIPE GERALDI-***628749**': ['44 99640852'],
 'VAGNER JOAQUIM FERREIRA DA SILVA-***437310**': ['44 99607562'],
 'LEANDRO BRAIDO MACRI-***795239**': ['44 33547787'],
 'LETYCIA RODRIGUES CZEZACKI-***263519**': ['44 88022643', '44 30276742'],
 'VINICIUS MENARIM HUMAMOTO-***649229**': ['43 33445115'],
 'FABRICIA BRASILINO DE SOUZA RODRIGUES-***576729**': ['43 98311055',
  '44 98311043',
  '14 32321473',
  '44 99469699'],
 'EDNALDO CARLOS ROZAO-***515849**': ['44 99944577'],
 'NILZA VALERIA DE ALMEIDA-***217638**': ['44 30252281',
  '44 30262927',
  '44 98500121'],
 'WILLIANS PEREIRA DA SILVA-***851769**': ['44 91812016'],
 'JOAO PEDRO CARREGOSA ANDRADE COSTA-***599239**': ['44 99585439'],
 'SANDRA MASSON UTINOI-***247979**': ['44 30316050'],
 'YASMIN FERRACINI FOUTO-***127279**': ['44 91613019'],
 'ELISSANDRA DA SILVA-***270199**': ['44 97613344'],
 'VALERIA SICHIERI GUILHEN-***069539**': ['44 98565985'],
 'PAOLA CRISTINA FRANCOLIN-***294779**': ['44 99708684'],
 'THIAGO CALIXTO OLIVEIR

In [ ]:
todos_telefones

['44 99640852',
 '44 99607562',
 '44 33547787',
 '44 88022643',
 '44 30276742',
 '43 33445115',
 '43 98311055',
 '44 98311043',
 '14 32321473',
 '44 99469699',
 '44 99944577',
 '44 30252281',
 '44 30262927',
 '44 98500121',
 '44 91812016',
 '44 99585439',
 '44 30316050',
 '44 91613019',
 '44 97613344',
 '44 98565985',
 '44 99708684',
 '44 30479697',
 '44 98632258']